# Prova NeoSemantics

https://towardsdatascience.com/traveling-tourist-part-1-import-wikidata-to-neo4j-with-neosemantics-library-f80235f40dc5

Settare i plugin di neosemantics

In [2]:
pip install graphdatascience

  Using cached graphdatascience-1.8-py3-none-any.whl (1.6 MB)
  Using cached multimethod-1.10-py3-none-any.whl (9.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
from graphdatascience import GraphDataScience
from neo4j import GraphDatabase



In [8]:

driver = GraphDatabase.driver(uri='bolt://localhost:7687', auth=('neo4j','simone98'))
session = driver.session()


Opzione di configurazione:

CALL n10s.graphconfig.init();


https://neo4j.com/labs/neosemantics/4.0/reference/

In [11]:

conf='''CALL n10s.graphconfig.init({handleVocabUris: "IGNORE"})'''
#session.run(conf)

Per le ontologie con indirizzi uri devoaggiungere un  vincolo


In [13]:
vincolo = '''CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource)
REQUIRE r.uri IS UNIQUE;'''
session.run(vincolo)

In [16]:
#cancella nodi e relazioni
delete ='''match (n) detach delete n'''
session.run(delete)

# Esempio  Wiki Data

In [ ]:
#Creare lo schema
q2='''CALL apoc.schema.assert(
  {State:['id'], City:['id']},
  {Resource:['uri']})'''# Richiamare schema
#session.run(q2)

In [ ]:
# #query paper che utilizza wikidata api per costruire il grafo
q3='''WITH 'PREFIX sch: <http://schema.org/> 
CONSTRUCT{ ?item a sch:Monument; 
            sch:name ?monumentName; 
            sch:location ?location; 
            sch:img ?imageAsStr; 
            sch:ARCHITECTURE ?architecture. 
           ?architecture a sch:Architecture; 
            sch:name ?architectureName } 
WHERE { ?item wdt:P31 wd:Q4989906 . 
        ?item wdt:P17 wd:Q29 . 
        ?item rdfs:label ?monumentName . 
         filter(lang(?monumentName) = "en") 
        ?item wdt:P625 ?location . 
        ?item wdt:P149 ?architecture . 
        ?architecture rdfs:label ?architectureName .
         filter(lang(?architectureName) = "en") 
        ?item wdt:P18 ?image . 
         bind(str(?image) as ?imageAsStr) }' AS sparql 
CALL n10s.rdf.import.fetch(
  "https://query.wikidata.org/sparql?query=" +   
   apoc.text.urlencode(sparql),"JSON-LD", 
   { headerParams: { Accept: "application/ld+json"} , 
     handleVocabUris: "IGNORE"}) 
YIELD terminationStatus, triplesLoaded
RETURN terminationStatus, triplesLoaded''' #Caricare RDF  da WikiData
#session.run(q3)


Ricorda che dopo aver cancellato i nodi e le relazioni per caricare grafo, devi inserire vincolo

In [ ]:
#Query modificata ma bisogna inserire lo schema
Q = '''WITH 'PREFIX sch: <http://schema.org/> 
CONSTRUCT {
  ?item a sch:MedicalCondition;
        sch:name ?diseaseName;
        sch:cause ?cause;
        sch:consequence ?consequence.
  ?cause a sch:MedicalCondition;
        sch:name ?causeName.
  ?consequence a sch:MedicalCondition;
              sch:name ?consequenceName.
}
WHERE {
  ?item wdt:P31 wd:Q12136;  # Malattia (Q12136)
        rdfs:label ?diseaseName.
  FILTER(LANG(?diseaseName) = "en").
  
  OPTIONAL { 
    ?item wdt:P828 ?causeItem. # Proprietà per la causa della malattia
    ?causeItem rdfs:label ?causeLabel.
    FILTER(LANG(?causeLabel) = "en").
    BIND(sch:cause AS ?cause).
  }

  OPTIONAL { 
    ?item wdt:P780 ?consequenceItem. # Proprietà per la conseguenza della malattia
    ?consequenceItem rdfs:label ?consequenceLabel.
    FILTER(LANG(?consequenceLabel) = "en").
    BIND(sch:consequence AS ?consequence).
  }
}' AS sparql 

CALL n10s.rdf.import.fetch(
  "https://query.wikidata.org/sparql?query=" +   
   apoc.text.urlencode(sparql),
  "JSON-LD", 
  { headerParams: { Accept: "application/ld+json"} , 
    handleVocabUris: "IGNORE"
  }) 

YIELD terminationStatus, triplesLoaded
RETURN terminationStatus, triplesLoaded;'''
session.run(Q)

Si possono scaricare i grafi da WikiData attraverso query SPARQL il problema è che i dat provengono da Wikipeida quindi come fonte non è affidabile in ambito medico

# Ontologie Bio Portal

In [17]:
brenda = '''CALL n10s.rdf.import.fetch('https://github.com/SimoneFarallo/Thesis/raw/main/Codes/KB/prova_owl.rdf',"RDF/XML")'''
session.run(brenda)

Quando carico nodi e relazioni, essi cambiano in base alla configurazine di prima:

https://neo4j.com/labs/neosemantics/4.0/config/

https://neo4j.com/labs/neosemantics/4.0/reference/

In [9]:
bionut = '''CALL n10s.rdf.import.fetch('https://github.com/SimoneFarallo/Thesis/raw/main/Codes/KB/bionutrition_cam9.owl',"RDF/XML")'''
session.run(bionut)

Non ho lo schema quindi non riescco ad estrarre i nomi